In [1]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&

In [2]:
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package w3m.
(Reading database ... 123074 files and directories currently installed.)
Preparing to unpack .../w3m_0.5.3-37ubuntu0.1_amd64.deb ...
Unpacking w3m (0.5.3-37ubuntu0.1) ...
Setting up w3m (0.5.3-37ubuntu0.1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for mime-support (3.64ubuntu1) ...
/content
/content/drive
/content
/
Access token retrieved correctly.


In [22]:
import os
import tensorflow.keras as keras

data: dict
epochs_ = 300
lr = 0.001
kernel = (2,2)
size_images = (192, 256)

In [4]:
import numpy as np

def index_genre(genre, genres):
    for (g,index) in zip(genres, range(len(genres))):
        if(g == genre):
            return index
    raise Exception(f"Not found {genre}")

def get_data_from_path(data_path,genres, decoder):
    data = {'in': [], 'out': []}
    for genre in genres:
        files = os.listdir(data_path + genre)
        for filename in files:
            filepath = data_path + genre + '/' + filename
            data['in'].append(decoder(filepath))
            data['out'].append(index_genre(genre, genres))

    data = {'in': np.array(data['in']),'out': np.array(data['out'])}

    return data

In [5]:
import cv2

def read_image(filepath):
    global size_images
    image = cv2.imread(filepath)
    return cv2.resize(image, size_images)

genres = ['blues', 'classical', 'country', 'disco',
          'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

data_path_train = '/content/drive/MyDrive/GTZAN/Ensemble_Data/MFCC/models/'
data_path_test = '/content/drive/MyDrive/GTZAN/Ensemble_Data/MFCC/test/'
data_path_validation = '/content/drive/MyDrive/GTZAN/Ensemble_Data/MFCC/validation/'

def get_data_train_mfcc():
    return get_data_from_path(data_path_train, genres, read_image)
def get_data_test_mfcc():
    return get_data_from_path(data_path_test, genres, read_image)
def get_data_validation_mfcc():
    return get_data_from_path(data_path_validation, genres, read_image)

In [23]:
training_data = get_data_train_mfcc()
print("Training data loaded, count = "+str(len(training_data['in'])))
v_data = get_data_validation_mfcc()
print("Validation data loaded, count = "+str(len(v_data['in'])))

def training():
    global epochs_
    global size_images
    global lr

    model = mfcc_cnn_model(input_shape=(
        size_images[1], size_images[0], 3), classes=10, filter_shape=kernel)

    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(),
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        metrics=['accuracy'])

    checkpoint = keras.callbacks.ModelCheckpoint(
        '/content/drive/MyDrive/mfcc_models/best_weights.h5',
        save_weights_only=True,
        save_best_only=True,
        monitor='val_accuracy',
        mode='max')

    model.fit(
        training_data['in'], training_data['out'],
        validation_data=(v_data['in'], v_data['out']),
        epochs=epochs_,
        callbacks=[checkpoint]
        )

    model.save('/content/drive/MyDrive/mfcc_models/mfcc_model.h5')


test_data = get_data_test_mfcc()
print("Testing data loaded, count = "+str(len(test_data['in'])))

def testing(model_name="mfcc_model.h5", epoch_name = 'best_weights.h5'):
    global size_images

    input = test_data['in']
    output = test_data['out']

    path = '/content/drive/MyDrive/mfcc_models/' + model_name
    model = keras.models.load_model(path)

    score = model.evaluate(input, output, verbose=0)
    # print('testing loss: ' + str(score[0]))
    print('testing accuracy: ' + str(score[1]))

    input = v_data['in']
    output = v_data['out']
    score = model.evaluate(input, output, verbose=0)
    # print('validation loss: ' + str(score[0]))
    print('validation accuracy: ' + str(score[1]))

    input = training_data['in']
    output = training_data['out']
    score = model.evaluate(input, output, verbose=0)
    # print('training loss: ' + str(score[0]))
    print('training accuracy: ' + str(score[1]))


Training data loaded, count = 557
Validation data loaded, count = 120
Testing data loaded, count = 120


In [24]:
def mfcc_cnn_model(input_shape=(128, 96, 1), classes=10, filter_shape=(3, 3)):
    X_input = keras.layers.Input(input_shape)

    X = keras.layers.Conv2D(32, filter_shape, activation='relu')(X_input)

    X = keras.layers.Conv2D(64, filter_shape, activation='relu')(X)
    X = keras.layers.AveragePooling2D((2, 2))(X)

    X = keras.layers.Conv2D(128, filter_shape, activation='relu')(X)
    X = keras.layers.AveragePooling2D((2, 2))(X)

    X = keras.layers.Conv2D(256, filter_shape, activation='relu')(X)
    X = keras.layers.GlobalAveragePooling2D()(X)

    X = keras.layers.Dense(256, activation='relu')(X)
    X = keras.layers.Dense(128, activation='relu')(X)
    X = keras.layers.Dense(64, activation='relu')(X)
    X = keras.layers.Dense(32, activation='relu')(X)
    X = keras.layers.Dense(classes, activation='softmax')(X)

    return keras.models.Model(inputs=X_input, outputs=X, name='SpectrogramCNN')

In [25]:
training()

Epoch 1/300
18/18 [==============================] - 7s 226ms/step - loss: 4.0428 - accuracy: 0.1005 - val_loss: 2.4364 - val_accuracy: 0.0667
Epoch 2/300
18/18 [==============================] - 4s 212ms/step - loss: 2.3449 - accuracy: 0.1275 - val_loss: 2.2953 - val_accuracy: 0.1333
Epoch 3/300
18/18 [==============================] - 4s 211ms/step - loss: 2.2701 - accuracy: 0.1490 - val_loss: 2.2983 - val_accuracy: 0.1833
Epoch 4/300
18/18 [==============================] - 2s 110ms/step - loss: 2.2245 - accuracy: 0.1598 - val_loss: 2.2942 - val_accuracy: 0.0750
Epoch 5/300
18/18 [==============================] - 4s 211ms/step - loss: 2.1966 - accuracy: 0.1616 - val_loss: 2.2553 - val_accuracy: 0.1917
Epoch 6/300
18/18 [==============================] - 4s 211ms/step - loss: 2.1706 - accuracy: 0.2029 - val_loss: 2.1993 - val_accuracy: 0.2417
Epoch 7/300
18/18 [==============================] - 2s 112ms/step - loss: 2.1401 - accuracy: 0.2316 - val_loss: 2.1797 - val_accuracy: 0.2250

In [26]:
#Save Weights in Model
def save_model(model_name="mfcc_model.h5", epoch_name = 'best_weights.h5'):
    path = '/content/drive/MyDrive/mfcc_models/' + model_name
    path_best_epoch = '/content/drive/MyDrive/mfcc_models/' + epoch_name
    model = keras.models.load_model(path)
    model.load_weights(path_best_epoch)
    model.save('/content/drive/MyDrive/mfcc_models/mfcc_model.h5')


In [27]:
save_model()
testing()

testing accuracy: 0.6166666746139526
validation accuracy: 0.6916666626930237
training accuracy: 0.6894075274467468
